In [8]:
import os
import json
import requests
import pandas as pd

import pytz
import time
from datetime import datetime

from dotenv import load_dotenv

In [9]:
load_dotenv()
TOKEN = os.getenv('JUSBRASIL_TOKEN')

In [10]:
def get_datetime_now(timezone="America/Sao_Paulo"):
    return datetime.now(tz=pytz.timezone("America/Sao_Paulo"))

In [11]:
def background_check(cpf, pagination=None, route_path='background_check/search', token=TOKEN):
    headers = {'Content-Type': 'application/json'}
    
    url = f'https://op.digesto.com.br/api/{route_path}'
    
    payload = {'document': f'{cpf.zfill(11)}'}
    if pagination and 'size' in pagination.keys() and 'cursor' in pagination.keys():
        payload.update(pagination)

    response = requests.request('POST', url, headers=headers, json=payload, auth=(f'{token}', ''))
    
    parsed = None
    print(response.status_code)
    if response.status_code == 200:
        parsed = json.loads(response.text)
    
    return parsed

In [12]:
def get_execution_time(parsed):
    execution_time, extensions_execution_time = -1, -1
    
    if parsed and parsed.get('extensions'):
        execution_time, extensions_execution_time = parsed.get('extensions').get('executionTime', -1), parsed.get('extensions').get('extensionsExecutionTime', -1)
        
    return execution_time, extensions_execution_time

In [13]:
def get_cursor(parsed):
    return parsed.get('data').get('root').get('crmLawsuitsByPersonId').get('pageInfo').get('endCursor')

In [14]:
def get_has_next_page(parsed):
    return parsed.get('data').get('root').get('crmLawsuitsByPersonId').get('pageInfo').get('hasNextPage')

In [15]:
def get_next_page_info(parsed, n_results_per_page = 10):
    assert(n_results_per_page <= 500)
    dict_pagination = None
    
    if parsed:
        cursor = get_cursor(parsed)

        if get_has_next_page(parsed):
            dict_pagination = {'pagination': {'size': n_results_per_page, 'cursor': cursor}}
        
    return dict_pagination

In [16]:
def normalize_driver_data(parsed):
    df = None
    if parsed:
        df = pd.json_normalize(parsed.get('data').get('root').get('crmLawsuitsByPersonId').get('edges'), sep='_', max_level=2)
    return df

In [17]:
def pipeline_search_driver_lawsuits(cpf, sleep_time_among_requests = 0.5):
    total_execution_time = 0
    total_extensions_execution_time = 0
    
    has_next_page = True
    last_cursor = None
    cursor = None
    dict_pagination = None
    
    df_all_data_driver = pd.DataFrame()

    page = 1
    while has_next_page:
        parsed = background_check(f'{cpf.zfill(11)}', pagination=dict_pagination)
        
        execution_time, extensions_execution_time = get_execution_time(parsed)
        total_execution_time += execution_time
        extensions_execution_time += extensions_execution_time
        
        df_page_data_driver = normalize_driver_data(parsed)
        
        df_page_data_driver['dt_search_for_lawsuits'] = pd.to_datetime(get_datetime_now().replace(tzinfo=None))
        df_page_data_driver['page'] = page
        df_page_data_driver['execution_time'] = execution_time
        df_page_data_driver['extensions_execution_time'] = extensions_execution_time
        
        df_all_data_driver = pd.concat([df_all_data_driver, df_page_data_driver], axis=0)

        dict_pagination = get_next_page_info(parsed, n_results_per_page=10)

        last_cursor = cursor
        cursor = get_cursor(parsed)
        has_next_page = get_has_next_page(parsed)

        if last_cursor == cursor:
            break
        
        page += 1
        time.sleep(sleep_time_among_requests)

    df_all_data_driver['total_execution_time'] = total_execution_time
    df_all_data_driver['total_extensions_execution_time'] = extensions_execution_time
    
    return df_all_data_driver.drop_duplicates()

In [18]:
def get_lawsuit_details(cnj, tipo_numero=5, route_path='tribproc/', token=TOKEN):
    headers = {'Content-Type': 'application/json'}
    
    url = f'https://op.digesto.com.br/api/{route_path}/{cnj}?tipo_numero={tipo_numero}'

    response = requests.request('GET', url, headers=headers, auth=(f'{token}', ''))
    
    parsed = None
    
    if response.status_code == 200:
        parsed = json.loads(response.text)
    
    return parsed

In [19]:
def normalize_lawsuit_data(parsed):
    df = None
    if parsed:
        df = pd.json_normalize(parsed, sep='_')
    return df

In [20]:
def pipeline_search_lawsuit_details(cnj):
    df_lawsuit_details = pd.DataFrame()
    
    lawsuit_details = get_lawsuit_details(f'{cnj.strip()}')
    if lawsuit_details:
        df_lawsuit_details = normalize_lawsuit_data(lawsuit_details)
        df_lawsuit_details['dt_search_lawsuit_details'] = pd.to_datetime(get_datetime_now().replace(tzinfo=None))
    
    return df_lawsuit_details

In [21]:
def pipeline_search_lawsuits_details(list_cnjs):
    df_lawsuits_details = pd.DataFrame()
    
    for cnj in list_cnjs:
        df_lawsuit_details = pipeline_search_lawsuit_details(cnj)
        df_lawsuits_details = pd.concat([df_lawsuits_details, df_lawsuit_details], axis=0)
    
    return df_lawsuits_details

In [22]:
df_cpfs = pd.read_csv('data/amostra_300.csv')
df_cpfs['cod_cpf'] = df_cpfs['cod_cpf'].astype(str).str.zfill(11)

In [24]:
cpfs = df_cpfs['cod_cpf'].values

cpfs_not_found = []
cpfs_not_found_details = []
df_all_drivers = pd.DataFrame()

start_time = time.time()
for cpf in cpfs: 
    print(f'{cpf}')
    df_driver_lawsuits = pipeline_search_driver_lawsuits(f'{cpf}')
    
    if not df_driver_lawsuits.empty:
        list_cnjs = df_driver_lawsuits['node_cnjNumber_number'].tolist()
        df_lawsuits_details = pipeline_search_lawsuits_details(list_cnjs)
        if not df_lawsuits_details.empty and 'numero' in df_lawsuits_details.columns:
            final_df = df_driver_lawsuits.merge(df_lawsuits_details, left_on='node_cnjNumber_number', right_on='numero')
            df_all_drivers = pd.concat([df_all_drivers, final_df], axis=0)
        else:
            cpfs_not_found_details.append(cpf)
    else:
        cpfs_not_found.append(cpf)

end_time = (time.time() - start_time)
print("--- %s seconds ---" % end_time)

df_all_drivers['total_exp_time'] = end_time
df_all_drivers.to_excel('drivers_lawsuits_details_300.xlsx', index=False)

df_all_drivers.head()

18263429234
200
46915001549
200
36903647368
200
99443228149
200
43323650115
200
79652760110
200
03820705694
200
00710417101
200
87732920104
200
27899225191
200
74410741691
200
10365284645
200
44662580153
200
57156417134
200
17112010144
200
14141353191
200
58049720144
200
89210352904
200
41585178187
200
76730263334
200
40536572100
200
27344517191
200


C:\Users\prove\AppData\Local\Temp\ipykernel_19396\2496197294.py:17: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_all_drivers = pd.concat([df_all_drivers, final_df], axis=0)


15030504915
200
33989923900
200
01454470909
200
200


C:\Users\prove\AppData\Local\Temp\ipykernel_19396\2496197294.py:17: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_all_drivers = pd.concat([df_all_drivers, final_df], axis=0)


30441501168
200
09512878933
200
40648842134
200
01822120926
200
02180410980
200
32497369968
200
77309740963
200
61802140930
200
05776720966
200
06490629909
200
05246898903
200
04240344921
200
02714919901
200
06549687993
200


...

C:\Users\prove\AppData\Local\Temp\ipykernel_19396\2496197294.py:17: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_all_drivers = pd.concat([df_all_drivers, final_df], axis=0)


07613928829
200
19510899879
200
10503623903
200
05874669876
200
79745512834
200
34652670877
200
37645173874
200
00632083182
200
03737940371
200
03710232546
200
00562286195
200


C:\Users\prove\AppData\Local\Temp\ipykernel_19396\2496197294.py:17: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_all_drivers = pd.concat([df_all_drivers, final_df], axis=0)


--- 410.6389174461365 seconds ---


ModuleNotFoundError: No module named 'openpyxl'

In [26]:
df_all_drivers['total_exp_time'] = end_time
df_all_drivers.to_excel('drivers_lawsuits_details_300.xlsx', index=False)

df_all_drivers.head()

,matchRelatedPeople_name,matchRelatedPeople_personId,matchRelatedPeople_role_normalized,matchRelatedPeople_role_rawValue,node_cnjNumber_number,node_court_rawValue,node_distributionDate,node_distributionType,node_district,node_forum,...,segredo_justica,arquivado,classes,acessos,uf,criadoEm,partes,dt_search_lawsuit_details,status_op,total_exp_time
0,Rubens da Silva Fernandes,x447043544110984535:24,Autor,AUTOR(A) DO FATO,5006597-20.2021.8.13.0035,TJMG,1633316400000,Sorteio,Araguari - Juizado Especial,None,...,False,False,[DIREITO PENAL / CRIMES PRATICADOS POR PARTICU...,[2021-10-05 03:59:08],MG,2021-10-05T03:59:08,"[[707289463, 8376968, AUTORIDADE POLICIAL, AUT...",2022-10-10 11:11:25.948199,NaN,410.638917
0,Jose Ferreira VAZ,x6998503631636920967:17,Requerido,Requerido,1000852-05.2021.8.11.0029,TJMT,1623898800000,None,Canarana,None,...,False,False,[Crimes de Trânsito],[2021-06-19 17:41:03],MT,2021-06-19T17:40:54,"[[666655748, 3936951, A COLETIVIDADE, A COLETI...",2022-10-10 11:11:35.850641,NaN,410.638917
0,Sabino Xavier dos Santos Junior,1464604427797647554:29,Réu,Polo Passivo,0005656-90.2022.8.16.0083,TJPR,1662692400000,None,Francisco Beltrão,Vara Criminal,...,False,False,"[Citação, Apropriação indébita]",[2022-09-09 21:24:28],PR,2022-09-09T21:24:28,"[[888809049, 19094, Ministério Público Estadua...",2022-10-10 11:11:41.122333,NaN,410.638917
1,Sabino Xavier dos Santos Junior,1464604427797647554:29,Réu,Polo Passivo,0005656-90.2022.8.16.0083,TJPR,1662692400000,None,Francisco Beltrão,Vara Criminal,...,False,False,"[Citação, Apropriação indébita]",[2022-09-09 21:24:28],PR,2022-09-09T21:24:28,"[[888809049, 19094, Ministério Público Estadua...",2022-10-10 11:11:48.185278,NaN,410.638917
2,Sabino Xavier dos Santos Junior,1464604427797647554:29,Réu,Polo Passivo,0005656-90.2022.8.16.0083,TJPR,1662692400000,None,Francisco Beltrão,Vara Criminal,...,False,False,"[Citação, Apropriação indébita]",[2022-09-09 21:24:28],PR,2022-09-09T21:24:28,"[[888809049, 19094, Ministério Público Estadua...",2022-10-10 11:11:41.122333,NaN,410.638917


In [ ]:
df_escavador = pd.read_csv('data/results_escavador.csv')

In [ ]:
df_escavador = df_escavador[['cod_cpf', 'result_escavador_manual']]

In [ ]:
df_escavador['cod_cpf'] = df_escavador['cod_cpf'].astype(str).str.zfill(11)
df_escavador['result'] = df_escavador['result_escavador_manual'].apply(lambda x: 1 if x == 'CRIMINAL' else 0)

In [ ]:
df_escavador.groupby('result').count()

In [ ]:
cpfs_encontrados_jusbrasil = set(cpfs) - set(cpfs_not_found + cpfs_not_found_details)

In [ ]:
## Casos encontrados pelo escavador, mas não encontrados pela Jusbrasil

In [ ]:
df_escavador[(df_escavador['result']==1)].shape[0], len(cpfs_encontrados_jusbrasil)

In [ ]:
df_escavador[(df_escavador['result']==1) & (df_escavador['cod_cpf'].isin(cpfs_encontrados_jusbrasil))]['cod_cpf']

In [ ]:
df_escavador[(df_escavador['result']==0) & (df_escavador['cod_cpf'].isin(cpfs_encontrados_jusbrasil))]['cod_cpf']

In [ ]:
df_escavador[(df_escavador['result']==1) & (~df_escavador['cod_cpf'].isin(cpfs_encontrados_jusbrasil))]['cod_cpf']

In [ ]:
cpfs_nao_encontrados_jusbrasil = df_escavador[(df_escavador['result']==1) & (~df_escavador['cod_cpf'].isin(cpfs_encontrados_jusbrasil))]['cod_cpf'].tolist()

In [ ]:
for cpf in cpfs_nao_encontrados_jusbrasil:
    parsed = background_check(cpf=cpf)
    if not parsed.get('data').get('root').get('crmLawsuitsByPersonId').get('edges'):
        print(f'{cpf} - nada encontrado.')